In [ ]:
#after you run this, output file is in tensorboard so use cmd :

'''tensorboard --logdir=runs'''

#This opens localhost port 6006, where you can see the output

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import os

In [4]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)


device = "cuda" if torch.cuda.is_available() else "cpu"

alpha = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1  # 784
batch_size = 32
num_epochs = 50

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5))]
)
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=alpha)
opt_gen = optim.Adam(gen.parameters(), lr=alpha)
criterion = nn.BCELoss()
log_dir = "runs/GAN_MNIST"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

writer_fake = SummaryWriter(os.path.join(log_dir, "fake"))
writer_real = SummaryWriter(os.path.join(log_dir, "real"))
step = 0

for epoch in range(num_epochs):
    for batch_index, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))

        disc_fake = disc(fake.detach()).view(-1) 
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_index == 0:
            print(f"Epoch [{epoch}/{num_epochs}] | LossD: {lossD:.4f}, LossG: {lossG:.4f}")

        if step % 100 == 0:
            with torch.no_grad():
                fake_images = gen(fixed_noise).reshape(-1, 1, 28, 28)
                real_images = real.reshape(-1, 1, 28, 28)

                img_grid_fake = torchvision.utils.make_grid(fake_images, normalize=True)
                img_grid_real = torchvision.utils.make_grid(real_images, normalize=True)

                writer_fake.add_image("MNIST Fake", img_grid_fake, global_step=step)
                writer_real.add_image("MNIST Real", img_grid_real, global_step=step)
            writer_fake.flush()
            writer_real.flush()

        step += 1

writer_fake.close()
writer_real.close()

Epoch [0/50] | LossD: 0.7671, LossG: 0.7343
Epoch [1/50] | LossD: 0.5095, LossG: 1.2067
Epoch [2/50] | LossD: 0.3994, LossG: 1.3423
Epoch [3/50] | LossD: 0.4208, LossG: 1.2068
Epoch [4/50] | LossD: 1.0245, LossG: 0.4982
Epoch [5/50] | LossD: 0.8006, LossG: 0.7448
Epoch [6/50] | LossD: 0.5252, LossG: 1.4189
Epoch [7/50] | LossD: 0.5968, LossG: 1.0558
Epoch [8/50] | LossD: 0.5415, LossG: 1.1630
Epoch [9/50] | LossD: 0.5643, LossG: 1.4379
Epoch [10/50] | LossD: 0.3726, LossG: 1.3754
Epoch [11/50] | LossD: 0.5218, LossG: 1.4200
Epoch [12/50] | LossD: 0.4234, LossG: 1.8057
Epoch [13/50] | LossD: 0.6230, LossG: 1.4258
Epoch [14/50] | LossD: 0.6816, LossG: 1.2001
Epoch [15/50] | LossD: 0.7385, LossG: 1.1236
Epoch [16/50] | LossD: 0.5696, LossG: 1.0894
Epoch [17/50] | LossD: 0.7684, LossG: 1.3227
Epoch [18/50] | LossD: 0.4629, LossG: 1.5902
Epoch [19/50] | LossD: 0.8363, LossG: 1.2733
Epoch [20/50] | LossD: 0.3986, LossG: 1.7148
Epoch [21/50] | LossD: 0.4709, LossG: 1.2836
Epoch [22/50] | Loss

In [10]:
!ls /kaggle/working/runs/GAN_MNIST/
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/runs/GAN_MNIST

fake  real
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 91), started 0:00:26 ago. (Use '!kill 91' to kill it.)

<IPython.core.display.Javascript object>

ls: cannot access '/kaggle/output/': No such file or directory


In [4]:
!nvidia-smi

Mon Mar 17 02:48:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             31W /  250W |     335MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----